In [1]:

# import SparkSession
from pyspark.sql import SparkSession
import zipfile
import os

In [2]:
spark = SparkSession.builder\
          .master("local[10]")\
          .appName("Colab")\
          .config('spark.ui.port','4050')\
          .getOrCreate()

22/11/03 15:41:18 WARN Utils: Your hostname, MacBook-Pro-de-Jhon.local resolves to a loopback address: 127.0.0.1; using 192.168.1.1 instead (on interface en0)
22/11/03 15:41:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/03 15:41:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!pwd

/Users/sebastian/ApacheSpark/ipynb


In [3]:
!mkdir ~/.kaggle

mkdir: /Users/sebastian/.kaggle: File exists


In [4]:
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download mkechinov/ecommerce-events-history-in-cosmetics-shop

ecommerce-events-history-in-cosmetics-shop.zip: Skipping, found more recently modified local copy (use --force to force download)


In [10]:
for file in os.listdir():
  print(file)
  if file.endswith('.zip'):
    zip_ref = zipfile.ZipFile(file,'r')
    zip_ref.extractall()
    zip_ref.close()

2020-Jan.csv
2019-Dec.csv
.DS_Store
kaggle.json
ecommerce-events-history-in-cosmetics-shop.zip
2020-Feb.csv
2019-Nov.csv
.ipynb_checkpoints
pyspark.ipynb
2019-Oct.csv


In [3]:
df= spark.read.options(header='True',inferScheme='True').csv('*.csv')

In [8]:
df.count()

20692840

In [13]:
df.printSchema()

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)



In [10]:
df.select("event_type").distinct().show()

+----------------+
|      event_type|
+----------------+
|        purchase|
|            view|
|            cart|
|remove_from_cart|
+----------------+



In [11]:
df.select("brand").distinct().show()

+------------+
|       brand|
+------------+
|     beautix|
|     farmona|
|  dr.gloderm|
|   profhenna|
|     philips|
|invisibobble|
|       riche|
|        oniq|
|    lebelage|
|     vilenta|
|       fancy|
|      jaguar|
|      tertio|
|    siberina|
|   koreatida|
|         jas|
|rocknailstar|
|   depilflax|
|protokeratin|
|       essie|
+------------+
only showing top 20 rows



SQL

In [9]:
df.createOrReplaceTempView("data")
spark.sql("select * from data limit 3").show()

results = spark.sql("select * from data")
results.map(lambda x: x.product_id).show()



+--------------------+----------------+----------+-------------------+-------------+------+-----+---------+--------------------+
|          event_time|      event_type|product_id|        category_id|category_code| brand|price|  user_id|        user_session|
+--------------------+----------------+----------+-------------------+-------------+------+-----+---------+--------------------+
|2019-12-01 00:00:...|remove_from_cart|   5712790|1487580005268456287|         null| f.o.x| 6.27|576802932|51d85cb0-897f-48d...|
|2019-12-01 00:00:...|            view|   5764655|1487580005411062629|         null|   cnd|29.05|412120092|8adff31e-2051-489...|
|2019-12-01 00:00:...|            cart|      4958|1487580009471148064|         null|runail| 1.19|494077766|c99a50e8-2fac-4c4...|
+--------------------+----------------+----------+-------------------+-------------+------+-----+---------+--------------------+



AttributeError: 'DataFrame' object has no attribute 'map'

In [17]:
df.select(['product_id']).filter("event_type='cart'").first()

Row(product_id='4958')

In [18]:
sessions = df.select(['user_session']).filter("event_type='cart' AND product_id=4958").distinct()

In [19]:
products = df.select(['product_id']).filter("event_type='cart' AND product_id<>4958").filter( df["user_session"].isin(sessions["user_session"]) )

In [20]:
products.count()

5759760

In [21]:
products = products.distinct()

In [22]:
products.count()

46156

In [23]:
products.write.mode("overwrite").csv('resultado')

In [24]:
!cd resultado && ls

_SUCCESS
part-00000-a74920b3-e4e3-46cd-ba78-3db7a9043934-c000.csv
part-00001-a74920b3-e4e3-46cd-ba78-3db7a9043934-c000.csv
part-00002-a74920b3-e4e3-46cd-ba78-3db7a9043934-c000.csv
part-00003-a74920b3-e4e3-46cd-ba78-3db7a9043934-c000.csv


streaming

rdd

In [25]:
df.select("brand").distinct().show()

+------------+
|       brand|
+------------+
|     beautix|
|     farmona|
|  dr.gloderm|
|   profhenna|
|     philips|
|invisibobble|
|       riche|
|        oniq|
|    lebelage|
|     vilenta|
|       fancy|
|      jaguar|
|      tertio|
|    siberina|
|   koreatida|
|         jas|
|rocknailstar|
|   depilflax|
|protokeratin|
|       essie|
+------------+
only showing top 20 rows



In [26]:
def myFunc(s):
  if s["brand"]=="riche" and s["event_type"]=="cart":
    return [ ( s["product_id"], 1) ]
  return []

In [27]:
lines=df.rdd.flatMap(myFunc).reduceByKey(lambda a, b: a + b)


In [28]:
for element in lines.collect(): 
  print(element)

('5842214', 13)
('5842215', 10)
('5842242', 7)
('5842212', 3)
('5842205', 12)
('5842220', 6)
('5922103', 1)
('5842213', 12)
('5842241', 6)
('5842219', 9)
('5842231', 16)
('5922106', 3)
('5842202', 17)
('5842203', 13)
('5922109', 1)
('5842233', 4)
('5922115', 1)
('5922123', 1)
('5922121', 1)
('5842232', 20)
('5842268', 131)
('5922107', 5)
('5922119', 2)
('5922110', 3)
('5922122', 1)
('5842235', 12)
('5842258', 20)
('5844573', 3)
('5922097', 1)
('5842223', 12)
('5846097', 5)
('5846098', 21)
('5922118', 9)
('5842224', 9)
('5922124', 1)
('5922112', 2)
('5922104', 2)
('5842221', 9)
('5842204', 17)
('5842217', 11)
('5842234', 5)
('5842206', 2)
('5844571', 20)
('5842240', 12)
('5922117', 6)
('5842222', 25)
('5842225', 6)
('5842226', 6)
('5842227', 11)
('5922114', 1)
('5922108', 1)


In [29]:
print( lines.take(20) )

[('5842214', 13), ('5842215', 10), ('5842242', 7), ('5842212', 3), ('5842205', 12), ('5842220', 6), ('5922103', 1), ('5842213', 12), ('5842241', 6), ('5842219', 9), ('5842231', 16), ('5922106', 3), ('5842202', 17), ('5842203', 13), ('5922109', 1), ('5842233', 4), ('5922115', 1), ('5922123', 1), ('5922121', 1), ('5842232', 20)]


In [30]:
lines.saveAsTextFile("resultado2")

In [33]:
rdd=spark.sparkContext.parallelize([1,2,3,4,5])

rddCollect = rdd.collect()
print("Number of Partitions: "+str(rdd.getNumPartitions()))
print("Action: First element: "+str(rdd.first()))
print(rddCollect)

emptyRDD = spark.sparkContext.emptyRDD()
emptyRDD2 = rdd=spark.sparkContext.parallelize([])

print(""+str(emptyRDD2.isEmpty()))

Number of Partitions: 10
Action: First element: 1
[1, 2, 3, 4, 5]
True


In [34]:
states = {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)

data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]

rdd = spark.sparkContext.parallelize(data)

def state_convert(code):
    return broadcastStates.value[code]

result = rdd.map(lambda x: (x[0],x[1],x[2],state_convert(x[3]))).collect()
print(result)

[('James', 'Smith', 'USA', 'California'), ('Michael', 'Rose', 'USA', 'New York'), ('Robert', 'Williams', 'USA', 'California'), ('Maria', 'Jones', 'USA', 'Florida')]


In [35]:
data = [('Project', 1),
('Gutenberg’s', 1),
('Alice’s', 1),
('Adventures', 1),
('in', 1),
('Wonderland', 1),
('Project', 1),
('Gutenberg’s', 1),
('Adventures', 1),
('in', 1),
('Wonderland', 1),
('Project', 1),
('Gutenberg’s', 1)]

rdd=spark.sparkContext.parallelize(data)

rdd2=rdd.reduceByKey(lambda a,b: a+b)
for element in rdd2.collect():
    print(element)

('Alice’s', 1)
('in', 2)
('Gutenberg’s', 3)
('Adventures', 2)
('Wonderland', 2)
('Project', 3)


Manipular DataFrame usando SQL